In [45]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install openpyxl
!pip install tqdm





[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\33676\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\33676\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\33676\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\33676\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 1.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\33676\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [3]:


# Lecture des fichiers Excel et ajout manuel du ticker
repertoire = "data" 
dataframes_dict = {}

for fichier in os.listdir(repertoire): 
    chemin_complet = os.path.join(repertoire, fichier)

    if fichier.endswith((".xls", ".xlsx")):  
        print(f"File found : {fichier}")

    try:
        worksheet = pd.read_excel(chemin_complet, sheet_name=None)

        for sheet_name, df in worksheet.items():
            if 'Ticker' not in df.columns:
                df['Ticker'] = sheet_name  
                print(f"Ticker added to {sheet_name}")
                dataframes_dict[f"{fichier}_{sheet_name}"] = df
            else:
                print(f"Column 'Ticker' already existed in {fichier}")

    except Exception as e:
        print(f"Error during the process {fichier} : {e}")



File found : Batch 1.xlsx
Ticker added to AAPL
Ticker added to COP
Ticker added to ETN
Ticker added to LLY
Ticker added to ORCL
Ticker added to LRCX
Ticker added to XOM
Ticker added to MU
Ticker added to ADI
Ticker added to PG
Ticker added to KLAC
Ticker added to MDT
Ticker added to MO
Ticker added to IBM
Ticker added to CVX
Ticker added to GE
Ticker added to ABT
Ticker added to BMY
Ticker added to TT
Ticker added to AMD
File found : Batch 2.xlsx
Ticker added to RTX
Ticker added to MRK
Ticker added to TXN
Ticker added to CAT
Ticker added to BA
Ticker added to TMO
Ticker added to DE
Ticker added to SYK
Ticker added to HON
Ticker added to DHR
Ticker added to AMAT
Ticker added to PFE
Ticker added to INTC
Ticker added to PPG
Ticker added to PH
Ticker added to CDNS
Ticker added to EMR
Ticker added to NOC
Ticker added to CL
File found : Batch 3.xlsx
Ticker added to ITW
Ticker added to ADSK
Ticker added to APD
Ticker added to CHD
Ticker added to BDX
Ticker added to PCAR
Ticker added to HPQ
Ti

In [4]:
# On renomme et sépare les variables par fréquence
monthly_list = []
yearly_list = []
quarterly_list = []
daily_tot_return_list = []
daily_vol_list = [] #on fait deux listes daily parce que les dates de début sont différentes

for name, df in dataframes_dict.items():
    rename_map = {
        df.columns.values[0]: "Date.1",
        df.columns.values[1]: "Total_Assets",
        df.columns.values[2]: "Common_Equity",
        df.columns.values[3]: "Cash_And_Investments",
        df.columns.values[4]: "R&D_Expenses",
        df.columns.values[5]: "Inventories",
        df.columns.values[6]: "Dividends_Paid",
        df.columns.values[7]: "Gross_Fixed_Assets",
        df.columns.values[8]: "Income_Before_Extra_Items",
        df.columns.values[9]: "Sales_Revenue",
        df.columns.values[10]: "Depreciation_Amortization",

        df.columns.values[12]: "Date.2",
        df.columns.values[13]: "Mkt_Cap_Yearly",
        df.columns.values[14]: "Shares_Outstanding_Yearly",

        df.columns.values[16]: "Date.3",
        df.columns.values[17]: "Net_Income",

        df.columns.values[19]: "Date.4",
        df.columns.values[20]: "Px_Bid",
        df.columns.values[21]: "Px_Ask",
        df.columns.values[22]: "Shares_Outstanding_Monthly",
        df.columns.values[23]: "Mkt_Cap_Monthly",

        df.columns.values[25]: "Date.5",
        df.columns.values[26]: "Px_Last",

        df.columns.values[28]: "Date.6",
        df.columns.values[29]: "Volume",
    }

    df.rename(columns=rename_map, inplace=True)

    try:
        df_yearly = df[[
            "Date.1", "Total_Assets", "Common_Equity", "Cash_And_Investments",
            "R&D_Expenses", "Inventories", "Dividends_Paid", "Gross_Fixed_Assets",
            "Income_Before_Extra_Items", "Sales_Revenue", "Depreciation_Amortization",
            "Mkt_Cap_Yearly", "Shares_Outstanding_Yearly"
        ]].copy()
        
        df_quarterly = df[["Date.3", "Net_Income"]].copy()
       
        df_monthly = df[[
            "Date.4", "Px_Bid", "Px_Ask", "Shares_Outstanding_Monthly", "Mkt_Cap_Monthly", 
        ]].copy()

        df_daily_tot_return = df[[
            "Date.5", "Px_Last"
        ]].copy()

        df_daily_vol= df[[
            "Date.6", "Volume"
        ]].copy()
        
        df_yearly['Ticker'] = df['Ticker'].iloc[0]
        df_monthly['Ticker'] = df['Ticker'].iloc[0]
        df_quarterly['Ticker'] = df['Ticker'].iloc[0]
        df_daily_tot_return['Ticker'] = df['Ticker'].iloc[0]
        df_daily_vol['Ticker'] = df['Ticker'].iloc[0]
        
        yearly_list.append(df_yearly)
        monthly_list.append(df_monthly)
        quarterly_list.append(df_quarterly)
        daily_tot_return_list.append(df_daily_tot_return)
        daily_vol_list.append(df_daily_vol)
        
        print(f"{name} : successful yearly / monthly / quarterly split")

    except Exception as e:
        print(f"{name} : error during split : {e}")

yearly_df = pd.concat(yearly_list, ignore_index=True)
monthly_df = pd.concat(monthly_list, ignore_index=True)
quarterly_df = pd.concat(quarterly_list, ignore_index=True)
daily_tot_return_df = pd.concat(daily_tot_return_list, ignore_index=True)
daily_vol_df = pd.concat(daily_vol_list, ignore_index=True)



Batch 1.xlsx_AAPL : successful yearly / monthly / quarterly split
Batch 1.xlsx_COP : successful yearly / monthly / quarterly split
Batch 1.xlsx_ETN : successful yearly / monthly / quarterly split
Batch 1.xlsx_LLY : successful yearly / monthly / quarterly split
Batch 1.xlsx_ORCL : successful yearly / monthly / quarterly split
Batch 1.xlsx_LRCX : successful yearly / monthly / quarterly split
Batch 1.xlsx_XOM : successful yearly / monthly / quarterly split
Batch 1.xlsx_MU : successful yearly / monthly / quarterly split
Batch 1.xlsx_ADI : successful yearly / monthly / quarterly split
Batch 1.xlsx_PG : successful yearly / monthly / quarterly split
Batch 1.xlsx_KLAC : successful yearly / monthly / quarterly split
Batch 1.xlsx_MDT : successful yearly / monthly / quarterly split
Batch 1.xlsx_MO : successful yearly / monthly / quarterly split
Batch 1.xlsx_IBM : successful yearly / monthly / quarterly split
Batch 1.xlsx_CVX : successful yearly / monthly / quarterly split
Batch 1.xlsx_GE : succes

In [ ]:
print(f"Type de yearly_df : {type(yearly_df)}")
print(f"Type de monthly_df : {type(monthly_df)}")
print(f"Type de daily_tot_return_df : {type(daily_tot_return_df)}")
print(f"Type de daily_vol_df : {type(daily_vol_df)}")
print(f"Type de quarterly_df : {type(quarterly_df)}")


Type de yearly_df : <class 'pandas.core.frame.DataFrame'>
Type de monthly_df : <class 'pandas.core.frame.DataFrame'>
Type de daily_tot_return_df : <class 'pandas.core.frame.DataFrame'>
Type de daily_vol_df : <class 'pandas.core.frame.DataFrame'>
Type de quarterly_df : <class 'pandas.core.frame.DataFrame'>


In [6]:
lists = {
    "yearly": yearly_df,
    "quarterly": quarterly_df,
    "monthly": monthly_df,
    "daily_tot_return": daily_tot_return_df,
    "daily_vol": daily_vol_df
}

# Itération sur les DataFrames
for name, df in lists.items():
    # Renommer les colonnes "Date.xxx" en "Date"
    df.rename(columns={col: "Date" for col in df.columns if col.startswith("Date")}, inplace=True)
    
    # Convertir "Date" en datetime
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    
    # Convertir toutes les autres colonnes en float64 (sauf "Date" et "Ticker")
    for col in df.columns:
        if col not in ["Date", "Ticker"]:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype("float64")

    # Affichage des informations pour vérification
    print(f"--- {name.capitalize()} DataFrame ---")
    print(f"  - Colonnes et Types:")
    for col in df.columns:
        print(f"    - {col}: {df[col].dtype}")
    print(f"  - Nombre de lignes: {df.shape[0]}")
    print(f"  - Nombre de colonnes: {df.shape[1]}")
    print("-" * 50)


--- Yearly DataFrame ---
  - Colonnes et Types:
    - Date: datetime64[ns]
    - Total_Assets: float64
    - Common_Equity: float64
    - Cash_And_Investments: float64
    - R&D_Expenses: float64
    - Inventories: float64
    - Dividends_Paid: float64
    - Gross_Fixed_Assets: float64
    - Income_Before_Extra_Items: float64
    - Sales_Revenue: float64
    - Depreciation_Amortization: float64
    - Mkt_Cap_Yearly: float64
    - Shares_Outstanding_Yearly: float64
    - Ticker: object
  - Nombre de lignes: 454310
  - Nombre de colonnes: 14
--------------------------------------------------
--- Quarterly DataFrame ---
  - Colonnes et Types:
    - Date: datetime64[ns]
    - Net_Income: float64
    - Ticker: object
  - Nombre de lignes: 454310
  - Nombre de colonnes: 3
--------------------------------------------------
--- Monthly DataFrame ---
  - Colonnes et Types:
    - Date: datetime64[ns]
    - Px_Bid: float64
    - Px_Ask: float64
    - Shares_Outstanding_Monthly: float64
    - Mkt_

In [7]:
#Cellules NaN

# Nettoyage : suppression des lignes où la colonne de date est NaN > "Lors de la séparation des données en fonction des fréquences, 
# chaque ligne obtient le Ticker correspondant. Cependant, comme nous avons plusieurs variables daily, chaque Ticker est dupliqué 
# autant de fois qu'il y a de lignes associées aux variables daily, ce qui entraîne une surabondance de lignes contenant uniquement 
# le Ticker dans les DataFrames yearly, quarterly et monthly, avec des cellules vides pour les autres colonnes."
lists["yearly"] = yearly_df.dropna(subset=["Date"])
lists["quarterly"] = quarterly_df.dropna(subset=["Date"])
lists["monthly"] = monthly_df.dropna(subset=["Date"])
lists["daily_vol"] = daily_vol_df.dropna(subset=["Date"])

all_nan_matrices = {}

for name, df in lists.items():  
    
    if "Ticker" in df.columns:
        # Calcul du % de NaN par colonne pour chaque ticker
        nan_matrix = df.groupby("Ticker").apply(lambda g: g.isna().mean() * 100)
        all_nan_matrices[name] = nan_matrix
        print(f"\n % Pourcentage de NaN pour {name} :\n", nan_matrix)


 % Pourcentage de NaN pour yearly :
         Date  Total_Assets  Common_Equity  Cash_And_Investments  R&D_Expenses  \
Ticker                                                                          
AAPL     0.0           0.0       0.000000              0.000000      0.000000   
ABT      0.0           0.0       0.000000              0.000000      0.000000   
ADI      0.0           0.0       0.000000              0.000000      0.000000   
ADSK     0.0           0.0       0.000000              0.000000      0.000000   
AMAT     0.0           0.0       0.000000              0.000000      0.000000   
AMD      0.0           0.0       0.000000              3.030303      0.000000   
APD      0.0           0.0       0.000000              0.000000      3.030303   
BA       0.0           0.0       0.000000              0.000000      0.000000   
BDX      0.0           0.0       0.000000              0.000000      0.000000   
BMY      0.0           0.0       0.000000              0.000000      0.

C:\Users\33676\AppData\Local\Temp\ipykernel_120872\2810960075.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nan_matrix = df.groupby("Ticker").apply(lambda g: g.isna().mean() * 100)
C:\Users\33676\AppData\Local\Temp\ipykernel_120872\2810960075.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nan_matrix = df.groupby("Ticker").apply(lambda g: g.isna().mean() * 100)
C:\Users\33676\AppData\Local\Temp\ipykernel_


 % Pourcentage de NaN pour quarterly :
         Date  Net_Income  Ticker
Ticker                          
AAPL     0.0    0.806452     0.0
ABT      0.0    0.000000     0.0
ADI      0.0    0.000000     0.0
ADSK     0.0    0.000000     0.0
AMAT     0.0    0.000000     0.0
AMD      0.0    0.000000     0.0
APD      0.0    0.806452     0.0
BA       0.0    1.612903     0.0
BDX      0.0    0.806452     0.0
BMY      0.0    0.000000     0.0
CAT      0.0    0.000000     0.0
CDNS     0.0    0.806452     0.0
CHD      0.0    0.806452     0.0
CL       0.0    0.000000     0.0
CMI      0.0    0.000000     0.0
COP      0.0    0.000000     0.0
CVX      0.0    0.000000     0.0
DE       0.0    0.000000     0.0
DHR      0.0    0.000000     0.0
EA       0.0    1.612903     0.0
ED       0.0    0.806452     0.0
EMR      0.0    0.806452     0.0
ETN      0.0    0.000000     0.0
F        0.0    0.000000     0.0
GE       0.0    1.612903     0.0
GIS      0.0    0.806452     0.0
GLW      0.0    1.612903     0.0
HO

C:\Users\33676\AppData\Local\Temp\ipykernel_120872\2810960075.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nan_matrix = df.groupby("Ticker").apply(lambda g: g.isna().mean() * 100)
C:\Users\33676\AppData\Local\Temp\ipykernel_120872\2810960075.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nan_matrix = df.groupby("Ticker").apply(lambda g: g.isna().mean() * 100)


In [8]:
# On traite les NaN > SimpleImputer
from sklearn.impute import SimpleImputer

# On crée un imputer pour remplacer les NaN par la moyenne
imputer = SimpleImputer(strategy="mean")

for name, df in lists.items():
    # On sélectionne les colonnes à imputer (toutes sauf "Date" et "Ticker")
    cols_to_impute = [col for col in df.columns if col not in ["Date", "Ticker"]]
    
    # Utiliser .loc pour éviter le SettingWithCopyWarning
    df.loc[:, cols_to_impute] = imputer.fit_transform(df[cols_to_impute])


In [9]:
yearly_df = lists["yearly"]
quarterly_df = lists["quarterly"]
monthly_df = lists["monthly"]
daily_vol_df = lists["daily_vol"]


In [10]:
#calculs yearly   

#mkt equity et shares outstanding ont été extrait avec une formule BDH et sont exprimées en millions → il faut multiplier par 10^6
yearly_df["Mkt_Cap_Yearly"] = yearly_df["Mkt_Cap_Yearly"] * 1e6
yearly_df["Shares_Outstanding_Yearly"] = yearly_df["Shares_Outstanding_Yearly"] * 1e6

In [11]:
#calculs yearly   


# Asset growth (agr)
yearly_df.loc[:, "agr"] = (
    yearly_df.groupby("Ticker")["Total_Assets"].shift(1) - 
    yearly_df.groupby("Ticker")["Total_Assets"].shift(2)
) / yearly_df.groupby("Ticker")["Total_Assets"].shift(1)

# Cash productivity (cashpr)
yearly_df.loc[:, "cashpr"] = (
    yearly_df["Mkt_Cap_Yearly"] - yearly_df["Common_Equity"]
) / yearly_df["Cash_And_Investments"]


# Change in inventory (chinv)
yearly_df.loc[:, "chinv"] = (
    yearly_df.groupby("Ticker")["Inventories"].shift(1) - 
    yearly_df["Inventories"]
) / yearly_df["Total_Assets"]


# Depreciation / Gross Fixed Assets (depr)
yearly_df.loc[:, "depr"] = (
    yearly_df["Depreciation_Amortization"] / 
    yearly_df["Gross_Fixed_Assets"]
)

# Dividends to Market Cap (dy)
yearly_df.loc[:, "dy"] = (
    yearly_df["Dividends_Paid"] / 
    yearly_df["Mkt_Cap_Yearly"]
)

# Earnings to Price (ep)
yearly_df.loc[:, "ep"] = (
    yearly_df["Income_Before_Extra_Items"] / 
    yearly_df["Mkt_Cap_Yearly"]
)

# Investment to assets (invest)
yearly_df.loc[:, "invest"] = (
    yearly_df.groupby("Ticker")["Gross_Fixed_Assets"].transform(lambda x: x - x.shift(1))
    + yearly_df.groupby("Ticker")["Inventories"].transform(lambda x: x - x.shift(1))
) / yearly_df.groupby("Ticker")["Total_Assets"].transform(lambda x: x.shift(1))


# R&D to Market Value of Equity (rd_mve)
yearly_df.loc[:, "rd_mve"] = (
    yearly_df["R&D_Expenses"] / 
    yearly_df["Mkt_Cap_Yearly"]
)

# Sales to Price (sp)
yearly_df.loc[:, "sp"] = (
    yearly_df["Sales_Revenue"] / 
    yearly_df["Mkt_Cap_Yearly"]
)


In [12]:
yearly_df.to_excel("yearly_df.xlsx", index=False)

In [14]:
#Vérif cohérence des données > yearly 

covariate_cols = ["agr", "cashpr", "chinv", "depr", "dy", "ep", "invest", "rd_mve", "sp"]

# Moyenne de chaque covariate
covariate_means = yearly_df[covariate_cols].mean()
print("Moyennes des covariates :")
print(covariate_means)

# Écart-type de chaque covariate
covariate_stds = yearly_df[covariate_cols].std()
print("\nÉcarts-types des covariates :")
print(covariate_stds)

Moyennes des covariates :
agr         0.069342
cashpr    124.939467
chinv      -0.004935
depr        0.104531
dy         -0.022139
ep          0.044248
invest      0.047150
rd_mve      0.051214
sp          0.865267
dtype: float64

Écarts-types des covariates :
agr          0.156092
cashpr    2483.932928
chinv        0.076170
depr         0.068795
dy           0.067775
ep           0.110050
invest       0.252321
rd_mve       0.146870
sp           1.120341
dtype: float64


In [ ]:
#calculs quarterly 

quarterly_df["delta_income"] = (
    quarterly_df["Net_Income"] - quarterly_df["Net_Income"].shift(1)
)

quarterly_df["direction"] = np.sign(quarterly_df["delta_income"])


def compute_nincr(direction_series):
    nincr = []
    count = 0
    prev = 0
    for d in direction_series:
        if d == prev and d != 0:
            count += 1
        elif d != 0:
            count = 1
        else:
            count = 0
        nincr.append(count * d if d != 0 else 0)
        prev = d if d != 0 else prev
    return nincr

# 3. Application groupée par Ticker
quarterly_df["nincr"] = (
    quarterly_df.groupby("Ticker")["direction"].transform(compute_nincr)
)

print(quarterly_df.head())


        Date    Net_Income Ticker  delta_income  direction  nincr
0 1989-12-29  1.248400e+08   AAPL           NaN        NaN    NaN
1 1990-03-30  1.318150e+08   AAPL  6.975006e+06        1.0    1.0
2 1990-06-29  1.197640e+08   AAPL -1.205100e+07       -1.0   -1.0
3 1990-09-28  9.847600e+07   AAPL -2.128800e+07       -1.0   -2.0
4 1990-12-28  1.505090e+08   AAPL  5.203300e+07        1.0    1.0


In [12]:
#calculs monthly

#on a besoin de px_last et volume pour calculer les covariate monthly 
# Merge pour Px_Last
monthly_df = monthly_df.merge(
    daily_tot_return_df[["Date", "Ticker", "Px_Last"]],
    on=["Date", "Ticker"],
    how="left"
)

# Merge pour Volume
monthly_df = monthly_df.merge(
    daily_vol_df[["Date", "Ticker", "Volume"]],
    on=["Date", "Ticker"],
    how="left"
)

# On trie pour que le ffill fonctionne bien
monthly_df = monthly_df.sort_values(["Ticker", "Date"])

# Pour Px_Last
monthly_df["Px_Last"] = monthly_df.groupby("Ticker")["Px_Last"].ffill()

# Pour Volume
monthly_df["Volume"] = monthly_df.groupby("Ticker")["Volume"].ffill()


In [13]:
print(monthly_df.columns)

Index(['Date', 'Px_Bid', 'Px_Ask', 'Shares_Outstanding_Monthly',
       'Mkt_Cap_Monthly', 'Ticker', 'Px_Last', 'Volume'],
      dtype='object')


In [ ]:
#Calculs covariates montlhy

#Bid-ask spread (baspread)
monthly_df.loc[:, "baspread"] = (
    monthly_df["Px_Ask"] - monthly_df["Px_Bid"]
) / ((monthly_df["Px_Ask"] + monthly_df["Px_Bid"]) / 2)


#dollar trading volume (dolvol
monthly_df.loc[:, "dolvol"] = (np.log(
    monthly_df.groupby("Ticker")["Volume"].shift(2) * 
    monthly_df.groupby("Ticker")["Px_Last"].shift(2))
)

#size (mvel1)
monthly_df.loc[:, "mvel1"] = np.log(monthly_df["Volume"] * monthly_df["Px_Last"])

#share turnover (turn)
monthly_df.loc[:, "turn"] = (monthly_df["Volume"] / monthly_df["Shares_Outstanding_Monthly"])

#Calculs monthly plus complexes : maxret, momentums : création d'un df pour les calculs

#on transforme la date journalière en mois
daily_tot_return_df["Month"] = daily_tot_return_df["Date"].dt.to_period("M")

monthly_calculations_df = daily_tot_return_df["Month"].copy()

# Calcul du rendement journalier
daily_tot_return_df["daily return"] = (
    (daily_tot_return_df["Px_Last"] - daily_tot_return_df.groupby("Ticker")["Px_Last"].transform("shift", 1))
    / daily_tot_return_df.groupby("Ticker")["Px_Last"].transform("shift", 1)
)   

#maxret 
maxret_df = (
    daily_tot_return_df.groupby(["Ticker", "Month"])["daily return"]
    .max()
    .reset_index()
    .rename(columns={"daily return": "maxret"})
)

#Return volatility retvol)
retvol_df = (
    daily_tot_return_df
    .groupby(["Ticker", "Month"])["daily return"]
    .std()
    .reset_index()
    .rename(columns={"daily return": "retvol"})
)

#Calculs des momentums

#on calcule le rendement composé mensuel
monthly_return_df = (
    daily_tot_return_df.groupby(["Ticker", "Month"])["daily return"]
    .apply(lambda x: (1 + x).prod() - 1)
    .reset_index()
    .rename(columns={"daily return": "monthly return"})
)

#12-month mommentum (mom12m)
monthly_return_df["mom12m"] = (
    monthly_return_df.groupby("Ticker")["monthly return"]
    .transform(lambda x: (1 + x.shift(1)).rolling(window=11, min_periods=11).apply(np.prod, raw=True) - 1)
)

#6-month momentum (mom6m)
monthly_return_df["mom6m"] = (
    monthly_return_df.groupby("Ticker")["monthly return"]
    .transform(lambda x: (1 + x.shift(1)).rolling(window=6, min_periods=6).apply(np.prod, raw=True) - 1)
)

#1-month momentum (mom1m)
monthly_return_df["mom1m"] = (
    monthly_return_df.groupby("Ticker")["monthly return"]
    .transform(lambda x: (1 + x.shift(1)).rolling(window=1, min_periods=1).apply(np.prod, raw=True) - 1)
)

#on merge tous les df de calculs ensembles
monthly_calculations_df = (
    monthly_return_df
    .merge(maxret_df, on=["Ticker", "Month"], how="left")
    .merge(retvol_df, on=["Ticker", "Month"], how="left")
)



In [48]:
#Calculs covariates montlhy → beta

#on a besoin des weekly returns pour calculer beta

if daily_tot_return_df.index.name != "Date":
    daily_tot_return_df.set_index("Date", inplace=True)
    print("Index defined on Date")
else:
    print("Date is already in the index")


weekly_prices = daily_tot_return_df.groupby("Ticker")["Px_Last"].resample("W").last().reset_index()

#calcul weekly returns

weekly_prices["weekly_return"] = (
    (weekly_prices["Px_Last"] - weekly_prices.groupby("Ticker")["Px_Last"].transform("shift", 1))
    / weekly_prices.groupby("Ticker")["Px_Last"].transform("shift", 1)
)   


pivot = weekly_prices.pivot(index="Date", columns="Ticker", values="weekly_return") #transposée : 1 ligne par date 
pivot["Market"] = pivot.mean(axis=1, skipna=True) #moyenne des returns par semaine



Date is already in the index


In [ ]:
#regression linéaire pour avoir les betas

from tqdm import tqdm
from sklearn.linear_model import LinearRegression

results = []

for ticker in tqdm(pivot.columns.drop("Market")):
    #on boucle sur chaque mois en commençant 3 ans après la première date, freq ="M" car on avance de mois en mois
    for current_month in pd.date_range(start=pivot.index.min() + pd.DateOffset(years=3), 
                                       end=pivot.index.max(), freq="ME"):

        #current_month est déjà un Timestamp
        end_date = current_month - pd.DateOffset(months=1)
        start_date = end_date - pd.DateOffset(years=3)

        #extrais les données hebdomadaires pour cette fenêtre de 3 ans, contenant les rendements de l’action et du marché.
        #qd j'auras la data complète je pourrais enlever .dropna 
        try:
            window = pivot.loc[start_date:end_date, [ticker, "Market"]].dropna()

        except KeyError:
            # Fallback si la date n'existe pas exactement
            window = pivot.loc[(pivot.index >= start_date) & (pivot.index <= end_date), [ticker, "Market"]].dropna()
        #idem le if peut s'enlever 
        if len(window) >= 52:  # au moins 52 semaines de données
            X = window["Market"].values.reshape(-1, 1) #x = variable explicative
            y = window[ticker].values #y = variable cible
            model = LinearRegression().fit(X, y) #regression linéaire
            beta = model.coef_[0]

            results.append({
                "Ticker": ticker,
                "Month": current_month.to_period("M"),
                "Beta": beta
            }) #on ajoute le résultat dans une liste 

beta_df = pd.DataFrame(results)



100%|██████████| 58/58 [06:29<00:00,  6.71s/it]

  Ticker    Month      Beta
0   AAPL  1991-01  1.378063
1   AAPL  1991-02  1.319593
2   AAPL  1991-03  1.279117
3   AAPL  1991-04  1.303981
4   AAPL  1991-05  1.286432


In [47]:
beta_df.to_excel("beta_df.xlsx", index=False)

In [ ]:
#calculs yearly   


# Asset growth (agr)
yearly_df.loc[:, "agr"] = (
    yearly_df.groupby("Ticker")["Total_Assets"].shift(1) - 
    yearly_df.groupby("Ticker")["Total_Assets"].shift(2)
) / yearly_df.groupby("Ticker")["Total_Assets"].shift(1)

# Cash productivity (cashpr)
yearly_df.loc[:, "cashpr"] = (
    yearly_df["Mkt_Cap_Yearly"] - yearly_df["Common_Equity"]
) / yearly_df["Cash_And_Investments"]


# Change in inventory (chinv)
yearly_df.loc[:, "chinv"] = (
    yearly_df.groupby("Ticker")["Inventories"].shift(1) - 
    yearly_df["Inventories"]
) / yearly_df["Total_Assets"]


# Depreciation / Gross Fixed Assets (depr)
yearly_df.loc[:, "depr"] = (
    yearly_df["Depreciation_Amortization"] / 
    yearly_df["Gross_Fixed_Assets"]
)

# Dividends to Market Cap (dy)
yearly_df.loc[:, "dy"] = (
    yearly_df["Dividends_Paid"] / 
    yearly_df["Mkt_Cap_Yearly"]
)

# Earnings to Price (ep)
yearly_df.loc[:, "ep"] = (
    yearly_df["Income_Before_Extra_Items"] / 
    yearly_df["Mkt_Cap_Yearly"]
)

# Investment to assets (invest)
yearly_df.loc[:, "invest"] = (
    yearly_df.groupby("Ticker")["Gross_Fixed_Assets"].transform(lambda x: x - x.shift(1))
    + yearly_df.groupby("Ticker")["Inventories"].transform(lambda x: x - x.shift(1))
) / yearly_df.groupby("Ticker")["Total_Assets"].transform(lambda x: x.shift(1))


# R&D to Market Value of Equity (rd_mve)
yearly_df.loc[:, "rd_mve"] = (
    yearly_df["R&D_Expenses"] / 
    yearly_df["Mkt_Cap_Yearly"]
)

# Sales to Price (sp)
yearly_df.loc[:, "sp"] = (
    yearly_df["Sales_Revenue"] / 
    yearly_df["Mkt_Cap_Yearly"]
)
